## **SETTINGS**

In [ ]:
import os
import glob

%cd ./parking-lot-detection

# 경로 설정
HOME = os.getcwd()
DATA = os.path.join(HOME, 'data')
YOLOV9 = os.path.join(HOME, 'yolov9')
YOLOV8 = os.path.join(HOME, 'yolov8')
YOLONAS = os.path.join(HOME, 'yolonas')

In [ ]:
%cd $HOME

## **DATA**

### 1. Download from Roboflow

In [ ]:
if not os.path.exists(DATA):
    os.makedirs(DATA)

In [ ]:
%cd $DATA

!curl -L "https://public.roboflow.com/ds/dYKPAQ9Uc4?key=1Knp1nWbcm" > data.zip; unzip data.zip; rm data.zip

### 2. Check data

In [ ]:
# 데이터 수 확인
train_path = os.path.join(DATA, 'train')
test_path = os.path.join(DATA, 'test')
valid_path = os.path.join(DATA, 'valid')

train_images = glob.glob(train_path + '/images/*.jpg')
test_images = glob.glob(test_path + '/images/*.jpg')
valid_images = glob.glob(valid_path + '/images/*.jpg')

print(f'train: {len(train_images)}, valid: {len(valid_images)}, test: {len(test_images)}')
print(f'total data: {len(train_images) + len(valid_images) + len(test_images)}')

In [ ]:
# label 파일 확인
train_label = glob.glob(train_path + '/labels/*.txt')
test_label = glob.glob(test_path + '/labels/*.txt')
valid_label = glob.glob(valid_path + '/labels/*.txt')

print(f'text files: {len(train_label)}, {len(valid_label)}, {len(test_label)}')

### 3. Update YAML

In [ ]:
%cd $DATA

import yaml

with open('data.yaml', 'r') as f:
    data_yaml = yaml.full_load(f)
    data_yaml['path'] = DATA
    data_yaml['train'] = './train/images'
    data_yaml['val'] = './valid/images'
    data_yaml['test'] = './test/images'

with open('my_data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

## **TRAIN FOR MODEL SELECTION**

### 1. YOLO v8

In [ ]:
%cd $YOLOV8
!pip install -U ultralytics
!pip install e .

In [ ]:
# Train
!yolo task=detect mode=train model=yolov8s.pt \
    data="../../data/my_data.yaml" \
    epochs=10 batch=16 imgsz=640 workers=2

In [ ]:
# Inference with Test set
!yolo predict \
    model='runs/detect/train/weights/best.pt' \
    source='../../data/test/images' show_labels=False

### 2. YOLO v9

In [ ]:
%cd $YOLOV9
! pip install -r requirements.txt -q

In [ ]:
# Train
! python train_dual.py \
    --batch-size 4 --epochs 10 --img 640 --device 0 --workers 2 \
    --data "../../data/my_data.yaml" \
    --weights 'https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt' \
    --cfg "models/detect/yolov9-c.yaml" \
    --hyp "data/hyps/hyp.scratch-high.yaml"

In [ ]:
# Inference with Test set
! python detect.py \
    --weights "runs/train/exp/weights/best.pt" \
    --source "../../data/test/images" \
    --hide-labels

### 3. YOLO v9 GELAN

In [ ]:
%cd $YOLOV9
# ! pip install -r requirements.txt -q 

In [ ]:
# Train
! python train.py \
    --batch-size 8 --epochs 10 --img 640 --device 0 --workers 2 \
    --data "../../data/my_data.yaml" \
    --weights "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt" \
    --cfg "models/detect/gelan-c.yaml" \
    --hyp "data/hyps/hyp.scratch-high.yaml"

In [ ]:
# Inference with Test set
!python detect.py \
    --weights "runs/train/exp/weights/best.pt" \
    --source "../../data/test/images" \
    --hide-labels

### 4. YOLO NAS

In [ ]:
%cd $YOLONAS
! pip install -qq super_gradients==3.7.1

In [ ]:
from super_gradients.training import Trainer
from super_gradients.training import models
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val

from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

In [ ]:
# Parameters

dataset_params = {
   'data_dir': DATA,
   'train_images_dir':'train/images',
   'train_labels_dir':'train/labels',
   'val_images_dir':'valid/images',
   'val_labels_dir':'valid/labels',
   'test_images_dir':'test/images',
   'test_labels_dir':'test/labels',
   'classes': ['space-empty', 'space-occupied']}

dataloader_params={
   'batch_size':2,  # 배치 사이즈 증가
   'num_workers':4  # num_workers 증가
}

train_params = {
    "max_epochs": 5,
    "initial_lr": 5e-4,
    "optimizer": "Adam",
    "loss": PPYoloELoss(
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [ ]:
train_data = coco_detection_yolo_format_train(
   dataset_params={
       'data_dir': dataset_params['data_dir'],
       'images_dir': dataset_params['train_images_dir'],
       'labels_dir': dataset_params['train_labels_dir'],
       'classes': dataset_params['classes'],
       'cache_annotations': False  
   },
   dataloader_params=dataloader_params
)

val_data = coco_detection_yolo_format_val(
   dataset_params={
       'data_dir': dataset_params['data_dir'],
       'images_dir': dataset_params['val_images_dir'],
       'labels_dir': dataset_params['val_labels_dir'],
       'classes': dataset_params['classes'],
       'cache_annotations': False 
   },
   dataloader_params=dataloader_params
)

test_data = coco_detection_yolo_format_val(
   dataset_params={
       'data_dir': dataset_params['data_dir'],
       'images_dir': dataset_params['test_images_dir'],
       'labels_dir': dataset_params['test_labels_dir'],
       'classes': dataset_params['classes'],
       'cache_annotations': False 
   },
   dataloader_params=dataloader_params
)

clear_output()

In [ ]:
trainer = Trainer(experiment_name='exp', ckpt_root_dir='weight')
model = models.get('yolo_nas_s', num_classes=len(dataset_params['classes']), pretrained_weights="coco")

In [ ]:
trainer.train(model=model, training_params=train_params, train_loader=train_data, valid_loader=val_data)

## **TRAIN FOR VIDEO DETECTION**

In [ ]:
%cd $YOLOV9
# ! pip install -r requirements.txt -q 

In [ ]:
# Train
# 실행 전에 code/yolov9/utils/augmentations.py의 ln 26, T를 변경하고 실행 

! python train.py \
    --batch-size 8 --epochs 30 --img 640 --device 0 --workers 2 \
    --data "../../data/my_data.yaml" \
    --weights "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt" \
    --cfg "models/detect/gelan-c.yaml" \
    --hyp "data/hyps/hyp.scratch-high-mod.yaml"

In [ ]:
# Detection

!python detect.py \
    --weights "runs/train/exp/weights/best.pt" \
    --source "https://youtu.be/nJFxMsYyGrU?si=vPo5vXcH0yotw3Sc" \
    --device 0 \
    --hide-labels